In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
print(check_output(["ls", "../MNIST_data/fashion_csv/"]).decode("utf8"))

fashion-mnist_test.csv
fashion-mnist_train.csv.tar.gz



In [2]:
from keras.utils import to_categorical
import gzip
import pandas as pd
import os


# 完整的csv格式的训练集太大, 无法上传至github, 因此保存压缩文件, 在此解压缩并写入同名csv.
train_file = gzip.GzipFile("../MNIST_data/fashion_csv/fashion-mnist_train.csv.tar.gz", "r")
with open("../MNIST_data/fashion_csv/fashion-mnist_train.csv", 'wb') as f:
    f.write(train_file.read())
data_train = pd.read_csv('../MNIST_data/fashion_csv/fashion-mnist_train.csv')
data_train = data_train.drop(index=len(data_train.iloc[:, 0])-1) # 不知为什么, 最后一行全为nan, 因此删去末行.
os.remove('../MNIST_data/fashion_csv/fashion-mnist_train.csv')
data_test = pd.read_csv('../MNIST_data/fashion_csv/fashion-mnist_test.csv')

# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("../MNIST_data/fashion/", one_hot=True)
# X_train, y_train = mnist.train.images, mnist.train.labels
# X_test, y_test = mnist.test.images, mnist.test.labels

Using TensorFlow backend.


Extracting ../MNIST_data/fashion/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/fashion/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/fashion/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/fashion/t10k-labels-idx1-ubyte.gz


In [3]:
X_train = np.array(data_train.iloc[:, 1:])
X_test = np.array(data_test.iloc[:, 1:])
y_train = to_categorical(np.array(data_train.iloc[:, 0]))
y_test = to_categorical(np.array(data_test.iloc[:, 0]))


img_rows, img_cols = 28, 28
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D

input_shape = (img_rows, img_cols, 1)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
__________

In [6]:
model.fit(X_train, y_train,
          batch_size=50,
          epochs=10,
          verbose=1,
          validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
model.save_weights('my_model_weights.h5')

Train on 55000 samples, validate on 10000 samples
Epoch 1/10
55000/55000 [==============================] - 64s 1ms/step - loss: 1.0891 - acc: 0.5894 - val_loss: 0.7341 - val_acc: 0.7265
Epoch 2/10
55000/55000 [==============================] - 32s 586us/step - loss: 0.7644 - acc: 0.7101 - val_loss: 0.6495 - val_acc: 0.7517
Epoch 3/10
55000/55000 [==============================] - 32s 584us/step - loss: 0.6842 - acc: 0.7365 - val_loss: 0.5881 - val_acc: 0.7738
Epoch 4/10
55000/55000 [==============================] - 32s 582us/step - loss: 0.6401 - acc: 0.7565 - val_loss: 0.5649 - val_acc: 0.7763
Epoch 5/10
55000/55000 [==============================] - 31s 560us/step - loss: 0.6061 - acc: 0.7686 - val_loss: 0.5408 - val_acc: 0.7904
Epoch 6/10
55000/55000 [==============================] - 31s 572us/step - loss: 0.5847 - acc: 0.7773 - val_loss: 0.5197 - val_acc: 0.8088
Epoch 7/10
55000/55000 [==============================] - 31s 567us/step - loss: 0.5679 - acc: 0.7857 - val_loss: 0.50

In [8]:
print('accuracy: ',score[1])
print('loss: ',score[0])

accuracy:  0.8257
loss:  0.466920206881


In [9]:
print(score)

[0.46692020688056945, 0.82569999999999999]


In [10]:
model.save("weights_input.h5")
from keras.models import load_model
model_l = load_model(r"./weights_input.h5")
import read_image
data = read_image.Dataset()   

(6500, 28, 28, 1)
共6500张图片


In [11]:
prediction = model_l.predict(data.images)

In [12]:
with open("PengZheng.txt", "w") as f:
    for i in range(prediction.shape[0]):
        f.write(str(np.argmax(prediction[i])))
        if i != 6499:
            f.write("\n")

In [13]:
res = []
with open("PengZheng.txt", "r") as f:
    for i in range(6500):
        res.append(int(f.readline()))
labels = []
with open("../../../test_label.txt", "r") as f:
    for i in range(6500):
        labels.append(int(f.readline()))

In [14]:
import numpy as np
sum(np.array(res) == np.array(labels)) / len(res)

0.74030769230769233